In [1]:
import numpy as np
import pandas as pd 
import ipaddress
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns', None)

In [2]:
creditcard_data = pd.read_csv(r"C:\Users\ephre\Documents\Fraud-Detection-for-E-Commerce-and-Banking-Transactions---AI-Solutions\Data\Cleaned_Data\Creditcard_cleaned_Data.csv")
fraud_data = pd.read_csv(r"C:\Users\ephre\Documents\Fraud-Detection-for-E-Commerce-and-Banking-Transactions---AI-Solutions\Data\Cleaned_Data\fraud_cleaned_Data.csv")
ip_to_country = pd.read_csv(r"C:\Users\ephre\Documents\Fraud-Detection-for-E-Commerce-and-Banking-Transactions---AI-Solutions\Data\Cleaned_Data\ip_to_country_cleaned_Data.csv")

In [3]:
creditcard_data.sample(7)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
181525,125301.0,1.844235,-0.648795,-0.712148,0.230325,-0.404145,-0.183833,-0.360366,-0.032304,1.003179,-0.056292,-1.037952,0.336972,0.338207,-0.092023,0.578351,0.534075,-0.637243,-0.365814,0.059436,0.059956,-0.227003,-0.807530,0.350916,0.468801,-0.619831,0.204369,-0.062678,-0.021946,107.80,0
105316,69663.0,-0.893100,1.314857,1.169805,1.397126,-0.368209,-0.202039,0.185825,0.527681,-0.281365,-0.187928,-0.622357,-0.048569,-1.036655,0.490958,0.259894,-1.164211,0.960494,-0.649203,0.608581,-0.078757,0.029692,0.249239,-0.034148,0.399384,-0.148717,-0.250188,0.169235,0.137246,4.99,0
106481,70181.0,-3.632364,3.257590,-1.603419,-0.179103,-1.754905,-0.424541,-1.700849,2.877066,-0.590821,-0.171477,0.537682,1.263543,-0.185087,1.448905,0.389032,1.051127,0.918502,0.236901,-0.213496,0.030025,-0.117225,-0.764687,0.282267,-0.394912,0.144958,0.141880,0.049521,-0.021121,12.99,0
139613,83563.0,-2.039010,1.473372,1.169225,-1.809458,0.152995,-0.095572,0.612730,-0.130859,0.930623,1.290146,1.380096,0.547335,-0.029638,-0.556928,0.219678,0.350621,-0.974317,-0.330868,-0.689024,0.382431,-0.158237,-0.007440,-0.025759,-0.289991,-0.339509,0.621896,-0.705795,-0.376912,0.77,0
153033,99669.0,-1.251908,1.014314,0.819341,-0.477793,-0.416903,0.258817,-0.580714,0.944833,0.998881,-1.376974,1.278968,-1.182038,2.495132,1.717848,-1.614105,0.654512,0.197941,0.989270,-0.302242,-0.288304,0.313953,0.940291,-0.137770,0.796263,-0.389885,0.475841,-0.255728,-0.009490,29.90,0
233702,148027.0,2.063424,-0.028949,-1.054717,0.415437,-0.124878,-1.213350,0.201040,-0.349794,0.502614,0.066213,-0.689906,0.576411,0.315965,0.225297,0.011429,-0.119802,-0.299345,-0.903827,0.141311,-0.210088,-0.286253,-0.676253,0.338983,0.053213,-0.290697,0.194619,-0.068911,-0.059442,0.99,0
123816,77245.0,1.473944,-1.207480,0.782711,-1.290754,-1.975642,-0.839538,-1.324498,-0.071601,-1.399822,1.459858,-0.735504,-1.598319,-0.713975,-0.235731,0.983784,-0.087889,0.449315,0.468265,-0.437483,-0.378632,-0.068010,0.128803,0.004333,0.342450,0.291543,-0.100113,0.041152,0.026568,24.40,0


In [4]:
fraud_data.sample(7)

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,hour_of_day,day_of_week,signup_month,signup_day,signup_hour,purchase_month,purchase_day,purchase_hour
94243,279388,2015-04-29 23:23:05,2015-06-03 23:09:38,78.0,GGVRTZSVAZEJS,Ads,IE,F,27,1.423896e+09,0,23,2,4,29,23,6,3,23
12671,224316,2015-03-24 06:55:14,2015-04-24 02:35:09,32.0,IMUIJAUWNNGOA,Ads,Safari,F,25,1.121378e+09,0,2,4,3,24,6,4,24,2
82931,314775,2015-05-29 12:04:39,2015-07-03 13:59:10,52.0,UMMNFSNUDJQOL,Ads,Chrome,F,43,3.923263e+09,0,13,4,5,29,12,7,3,13
44055,324410,2015-04-17 04:35:15,2015-08-03 12:46:20,50.0,YWUTXRCYTOQDZ,Ads,Safari,M,44,3.787241e+09,0,12,0,4,17,4,8,3,12
22539,179675,2015-07-16 06:23:04,2015-11-11 12:21:25,52.0,QRNBVQWJYBNUR,Ads,Chrome,M,32,3.092299e+09,0,12,2,7,16,6,11,11,12
91691,342584,2015-05-03 03:31:02,2015-06-10 01:59:44,50.0,SQLIVGOVJHPMB,SEO,Chrome,M,26,2.011087e+09,0,1,2,5,3,3,6,10,1
29923,175922,2015-03-08 01:41:16,2015-03-09 04:29:37,14.0,FIIUFRUDBGJNU,SEO,IE,F,28,1.640595e+09,0,4,0,3,8,1,3,9,4


In [5]:
ip_to_country.sample(7)

,lower_bound_ip_address,upper_bound_ip_address,country
63575,3.225632e+09,3225632511,United States
54417,3.104518e+09,3104519167,Russian Federation
853,9.870131e+07,98705407,Spain
121184,3.426023e+09,3426022911,United States
80499,3.250742e+09,3250742783,Russian Federation
113485,3.398630e+09,3398630911,Indonesia
33052,1.823203e+09,1823207423,United States


In [6]:
# Assuming fraud_data['ip_address'] is already numeric (integer representation)
ip_to_country['lower_bound_ip_address'] = ip_to_country['lower_bound_ip_address'].astype(float).astype(int)
ip_to_country['upper_bound_ip_address'] = ip_to_country['upper_bound_ip_address'].astype(float).astype(int)

# Now, define a function to map IP to a country
def map_country(ip, ip_df):
    result = ip_df[(ip_df['lower_bound_ip_address'] <= ip) & (ip_df['upper_bound_ip_address'] >= ip)]
    return result['country'].values[0] if len(result) > 0 else np.nan

# Apply the function to map the IP addresses to their respective countries
fraud_data['country'] = fraud_data['ip_address'].apply(lambda x: map_country(x, ip_to_country))


In [7]:
import ipaddress

# Convert the ip_address column in fraud_data to integers
fraud_data['ip_address'] = fraud_data['ip_address'].apply(lambda x: int(ipaddress.ip_address(x)) if isinstance(x, str) else x)

# Convert the lower and upper IP address bounds in the ip_to_country DataFrame to integers
ip_to_country['lower_bound_ip_address'] = ip_to_country['lower_bound_ip_address'].apply(lambda x: int(ipaddress.ip_address(x)) if isinstance(x, str) else x)
ip_to_country['upper_bound_ip_address'] = ip_to_country['upper_bound_ip_address'].apply(lambda x: int(ipaddress.ip_address(x)) if isinstance(x, str) else x)

# Now map the country based on IP address
fraud_data['country'] = fraud_data['ip_address'].apply(lambda x: map_country(x, ip_to_country))

In [8]:
fraud_data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,hour_of_day,day_of_week,signup_month,signup_day,signup_hour,purchase_month,purchase_day,purchase_hour,country
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34.0,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,2,5,2,24,22,4,18,2,Japan
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16.0,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,1,0,6,7,20,6,8,1,United States
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15.0,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,18,3,1,1,18,1,1,18,United States
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44.0,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,13,0,4,28,21,5,4,13,NaN
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39.0,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,18,2,7,21,7,9,9,18,United States


In [9]:
fraud_data['country'] = fraud_data['ip_address'].apply(lambda x: map_country(x, ip_to_country))

creditcard_data.rename(columns={'Class': 'Class_creditcard'}, inplace=True)
fraud_data.rename(columns={'class': 'class_fraud'}, inplace=True)

missing_cols_in_fraud = set(creditcard_data.columns) - set(fraud_data.columns)
missing_cols_in_credit = set(fraud_data.columns) - set(creditcard_data.columns)

for col in missing_cols_in_fraud:
    fraud_data[col] = np.nan
for col in missing_cols_in_credit:
    creditcard_data[col] = np.nan

combined_data = pd.concat([fraud_data, creditcard_data], ignore_index=True)

# Inspect the merged dataset
combined_data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class_fraud,hour_of_day,day_of_week,signup_month,signup_day,signup_hour,purchase_month,purchase_day,purchase_hour,country,V4,V15,V18,V16,V6,V28,Class_creditcard,V21,V17,V23,V7,V25,V1,V10,V24,V14,Amount,V12,V20,V3,V8,V22,V5,V27,V2,V26,V19,Time,V9,V11,V13
0,22058.0,2015-02-24 22:55:49,2015-04-18 02:47:11,34.0,QVPSPJUOCKZAR,SEO,Chrome,M,39.0,7.327584e+08,0.0,2.0,5.0,2.0,24.0,22.0,4.0,18.0,2.0,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,333320.0,2015-06-07 20:39:50,2015-06-08 01:38:54,16.0,EOGFQPIZPYXFZ,Ads,Chrome,F,53.0,3.503114e+08,0.0,1.0,0.0,6.0,7.0,20.0,6.0,8.0,1.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1359.0,2015-01-01 18:52:44,2015-01-01 18:52:45,15.0,YSSKYOSJHPPLJ,SEO,Opera,M,53.0,2.621474e+09,1.0,18.0,3.0,1.0,1.0,18.0,1.0,1.0,18.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,150084.0,2015-04-28 21:13:25,2015-05-04 13:54:50,44.0,ATGTXKYKUDUQN,SEO,Safari,M,41.0,3.840542e+09,0.0,13.0,0.0,4.0,28.0,21.0,5.0,4.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,221365.0,2015-07-21 07:09:52,2015-09-09 18:40:53,39.0,NAUITBZFJKHWW,Ads,Safari,M,45.0,4.155831e+08,0.0,18.0,2.0,7.0,21.0,7.0,9.0,9.0,18.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
combined_data.sample(10)

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class_fraud,hour_of_day,day_of_week,signup_month,signup_day,signup_hour,purchase_month,purchase_day,purchase_hour,country,V4,V15,V18,V16,V6,V28,Class_creditcard,V21,V17,V23,V7,V25,V1,V10,V24,V14,Amount,V12,V20,V3,V8,V22,V5,V27,V2,V26,V19,Time,V9,V11,V13
219087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.328273,1.154791,0.035825,-0.064931,-0.485644,-0.086165,0.0,-0.140824,1.355073,-0.195392,-0.799806,-1.162515,-1.657163,0.825010,-0.375941,-1.250445,0.00,0.488493,-0.112175,1.119338,0.120981,-0.029371,2.939441,0.218013,-0.468878,-0.057786,0.270435,52894.0,-1.400466,2.245151,0.427595
282841,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.067405,0.641822,0.233142,0.210099,0.106697,0.015153,0.0,0.158194,-0.499796,-0.181584,-0.037664,0.551666,1.033697,0.190225,-0.318182,0.767247,74.97,0.253360,-0.077871,0.169109,0.151611,0.292092,-0.093840,0.009928,-0.059268,-0.252303,-0.435061,79909.0,-0.096594,1.021172,-1.091876
414687,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.949113,-1.105889,-0.006405,0.367761,-0.199059,0.078233,0.0,-0.268785,-1.069822,-0.080199,0.935245,-0.360309,0.078519,-0.260666,-1.024743,0.108780,2.28,0.860162,0.046800,-0.097087,-0.034882,-0.589239,1.021346,0.247061,0.846562,0.181746,0.585355,161477.0,-0.339704,-0.021069,0.800449
81778,74812.0,2015-01-27 12:54:59,2015-03-15 09:17:32,42.0,LEUHILAUUGFZY,SEO,Safari,F,33.0,3.920886e+09,0.0,9.0,6.0,1.0,27.0,12.0,3.0,15.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
266675,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.040771,0.128634,0.335578,0.738339,-0.920528,0.012277,0.0,-0.134529,0.147838,0.115867,-0.119585,-0.128627,-1.524194,-0.371350,0.483663,0.295819,14.99,1.037631,-0.002206,0.501864,0.877855,-0.586441,-0.667960,-0.020032,1.689025,0.038766,0.088077,74118.0,-0.744829,1.416888,0.182813
88681,384283.0,2015-03-21 10:32:43,2015-05-05 15:55:47,11.0,EBEROSOOGGTQJ,Ads,Safari,M,39.0,1.244365e+09,0.0,15.0,1.0,3.0,21.0,10.0,5.0,5.0,15.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24798,248106.0,2015-05-20 03:06:17,2015-09-04 06:01:07,33.0,BUFBXTRFUBGXO,Ads,Safari,F,35.0,1.431508e+09,0.0,6.0,4.0,5.0,20.0,3.0,9.0,4.0,6.0,Denmark,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
281018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.308884,-0.500598,-2.575471,0.957974,-1.328119,0.048671,0.0,-0.039794,0.254300,-0.076157,0.828457,0.432313,-0.904261,-0.489730,0.562004,-0.094975,23.90,-0.448396,-0.028985,1.724703,-0.161091,-0.419023,0.050273,-0.026031,0.307545,-0.727584,-1.267064,79345.0,-1.749674,0.257119,-0.044564
100884,57606.0,2015-04-19 19:11:04,2015-07-09 19:39:57,9.0,KIBYAAJGBTDLS,Ads,Safari,F,34.0,3.863105e+09,0.0,19.0,3.0,4.0,19.0,19.0,7.0,9.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
374777,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.071583,0.397761,-0.500113,0.208352,-1.040428,-0.044591,0.0,0.029048,-0.316345,0.158055,-0.147448,-0.471259,1.827193,-0.091871,0.123884,0.048679,126.32,0.275061,0.066220,-0.847914,-0.255327,-0.033629,-0.768242,-0.126022,-0.796064,1.143338,0.074444,143822.0,0.986605,-0.780120,-0.004463


In [11]:
# Drop columns with more than 65% missing values
combined_data = combined_data.dropna(thresh=combined_data.shape[0] * 0.20, axis=1)


In [12]:
# Example of filling missing values
combined_data['country'].fillna('Unknown', inplace=True)  # For categorical columns

C:\Users\ephre\AppData\Local\Temp\ipykernel_27844\3989163506.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_data['country'].fillna('Unknown', inplace=True)  # For categorical columns


In [13]:
# Fill missing values in numerical columns with the mean
combined_data['age'] = combined_data['age'].fillna(combined_data['age'].mean())

# Fill missing values in categorical columns with the mode
combined_data['sex'] = combined_data['sex'].fillna(combined_data['sex'].mode()[0])
combined_data['browser'] = combined_data['browser'].fillna(combined_data['browser'].mode()[0])
combined_data['Amount'] = combined_data['Amount'].fillna(combined_data['Amount'].mode()[0])


In [14]:
numerical_columns = ["purchase_value", "age", "class_fraud", "Class_creditcard", "V3", "V27", "V14", "V18", "V10", "V21", "V8", "V23", "V1", "V5", "V19", "V22", "V17", "V16", "V9", "V12", "V2", "V26", "V15", "V7", "V24", "V13", "V25", "V20", "V11", "V28", "V4", "V6", "purchase_month", "purchase_day", "purchase_hour"]

# Apply mean filling to all numerical columns
for col in numerical_columns:
    combined_data[col] = combined_data[col].fillna(combined_data[col].mean())

# Check the resulting DataFrame
combined_data.head()


,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class_fraud,hour_of_day,day_of_week,signup_month,signup_day,signup_hour,purchase_month,purchase_day,purchase_hour,country,V4,V15,V18,V16,V6,V28,Class_creditcard,V21,V17,V23,V7,V25,V1,V10,V24,V14,Amount,V12,V20,V3,V8,V22,V5,V27,V2,V26,V19,Time,V9,V11,V13
0,22058.0,2015-02-24 22:55:49,2015-04-18 02:47:11,34.0,QVPSPJUOCKZAR,SEO,Chrome,M,39.0,7.327584e+08,0.0,2.0,5.0,2.0,24.0,22.0,4.0,18.0,2.0,Japan,-0.002966,0.001043,0.001515,0.001162,-0.001139,0.000547,0.001667,-0.000371,0.00017,0.000198,0.001801,-0.000232,0.005917,-0.001441,0.000214,0.000252,1.0,-0.000715,0.000187,0.001613,-0.000854,-0.000015,0.001828,0.001763,-0.004135,0.000149,-0.000264,NaN,-0.001596,0.000202,0.000603
1,333320.0,2015-06-07 20:39:50,2015-06-08 01:38:54,16.0,EOGFQPIZPYXFZ,Ads,Chrome,F,53.0,3.503114e+08,0.0,1.0,0.0,6.0,7.0,20.0,6.0,8.0,1.0,United States,-0.002966,0.001043,0.001515,0.001162,-0.001139,0.000547,0.001667,-0.000371,0.00017,0.000198,0.001801,-0.000232,0.005917,-0.001441,0.000214,0.000252,1.0,-0.000715,0.000187,0.001613,-0.000854,-0.000015,0.001828,0.001763,-0.004135,0.000149,-0.000264,NaN,-0.001596,0.000202,0.000603
2,1359.0,2015-01-01 18:52:44,2015-01-01 18:52:45,15.0,YSSKYOSJHPPLJ,SEO,Opera,M,53.0,2.621474e+09,1.0,18.0,3.0,1.0,1.0,18.0,1.0,1.0,18.0,United States,-0.002966,0.001043,0.001515,0.001162,-0.001139,0.000547,0.001667,-0.000371,0.00017,0.000198,0.001801,-0.000232,0.005917,-0.001441,0.000214,0.000252,1.0,-0.000715,0.000187,0.001613,-0.000854,-0.000015,0.001828,0.001763,-0.004135,0.000149,-0.000264,NaN,-0.001596,0.000202,0.000603
3,150084.0,2015-04-28 21:13:25,2015-05-04 13:54:50,44.0,ATGTXKYKUDUQN,SEO,Safari,M,41.0,3.840542e+09,0.0,13.0,0.0,4.0,28.0,21.0,5.0,4.0,13.0,Unknown,-0.002966,0.001043,0.001515,0.001162,-0.001139,0.000547,0.001667,-0.000371,0.00017,0.000198,0.001801,-0.000232,0.005917,-0.001441,0.000214,0.000252,1.0,-0.000715,0.000187,0.001613,-0.000854,-0.000015,0.001828,0.001763,-0.004135,0.000149,-0.000264,NaN,-0.001596,0.000202,0.000603
4,221365.0,2015-07-21 07:09:52,2015-09-09 18:40:53,39.0,NAUITBZFJKHWW,Ads,Safari,M,45.0,4.155831e+08,0.0,18.0,2.0,7.0,21.0,7.0,9.0,9.0,18.0,United States,-0.002966,0.001043,0.001515,0.001162,-0.001139,0.000547,0.001667,-0.000371,0.00017,0.000198,0.001801,-0.000232,0.005917,-0.001441,0.000214,0.000252,1.0,-0.000715,0.000187,0.001613,-0.000854,-0.000015,0.001828,0.001763,-0.004135,0.000149,-0.000264,NaN,-0.001596,0.000202,0.000603


In [15]:
# List of columns you want to drop
columns_to_drop = ['user_id', 'signup_time', 'purchase_time', 'device_id', 'source', 'hour_of_day', 'day_of_week', 'signup_month', 'signup_day', 'signup_hour', 'Time', 'ip_address']

# Drop the columns from the DataFrame
combined_data = combined_data.drop(columns=columns_to_drop)

# Check the resulting DataFrame
combined_data.head()


,purchase_value,browser,sex,age,class_fraud,purchase_month,purchase_day,purchase_hour,country,V4,V15,V18,V16,V6,V28,Class_creditcard,V21,V17,V23,V7,V25,V1,V10,V24,V14,Amount,V12,V20,V3,V8,V22,V5,V27,V2,V26,V19,V9,V11,V13
0,34.0,Chrome,M,39.0,0.0,4.0,18.0,2.0,Japan,-0.002966,0.001043,0.001515,0.001162,-0.001139,0.000547,0.001667,-0.000371,0.00017,0.000198,0.001801,-0.000232,0.005917,-0.001441,0.000214,0.000252,1.0,-0.000715,0.000187,0.001613,-0.000854,-0.000015,0.001828,0.001763,-0.004135,0.000149,-0.000264,-0.001596,0.000202,0.000603
1,16.0,Chrome,F,53.0,0.0,6.0,8.0,1.0,United States,-0.002966,0.001043,0.001515,0.001162,-0.001139,0.000547,0.001667,-0.000371,0.00017,0.000198,0.001801,-0.000232,0.005917,-0.001441,0.000214,0.000252,1.0,-0.000715,0.000187,0.001613,-0.000854,-0.000015,0.001828,0.001763,-0.004135,0.000149,-0.000264,-0.001596,0.000202,0.000603
2,15.0,Opera,M,53.0,1.0,1.0,1.0,18.0,United States,-0.002966,0.001043,0.001515,0.001162,-0.001139,0.000547,0.001667,-0.000371,0.00017,0.000198,0.001801,-0.000232,0.005917,-0.001441,0.000214,0.000252,1.0,-0.000715,0.000187,0.001613,-0.000854,-0.000015,0.001828,0.001763,-0.004135,0.000149,-0.000264,-0.001596,0.000202,0.000603
3,44.0,Safari,M,41.0,0.0,5.0,4.0,13.0,Unknown,-0.002966,0.001043,0.001515,0.001162,-0.001139,0.000547,0.001667,-0.000371,0.00017,0.000198,0.001801,-0.000232,0.005917,-0.001441,0.000214,0.000252,1.0,-0.000715,0.000187,0.001613,-0.000854,-0.000015,0.001828,0.001763,-0.004135,0.000149,-0.000264,-0.001596,0.000202,0.000603
4,39.0,Safari,M,45.0,0.0,9.0,9.0,18.0,United States,-0.002966,0.001043,0.001515,0.001162,-0.001139,0.000547,0.001667,-0.000371,0.00017,0.000198,0.001801,-0.000232,0.005917,-0.001441,0.000214,0.000252,1.0,-0.000715,0.000187,0.001613,-0.000854,-0.000015,0.001828,0.001763,-0.004135,0.000149,-0.000264,-0.001596,0.000202,0.000603


In [16]:
# combined_data = combined_data.dropna()

In [17]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434838 entries, 0 to 434837
Data columns (total 39 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   purchase_value    434838 non-null  float64
 1   browser           434838 non-null  object 
 2   sex               434838 non-null  object 
 3   age               434838 non-null  float64
 4   class_fraud       434838 non-null  float64
 5   purchase_month    434838 non-null  float64
 6   purchase_day      434838 non-null  float64
 7   purchase_hour     434838 non-null  float64
 8   country           434838 non-null  object 
 9   V4                434838 non-null  float64
 10  V15               434838 non-null  float64
 11  V18               434838 non-null  float64
 12  V16               434838 non-null  float64
 13  V6                434838 non-null  float64
 14  V28               434838 non-null  float64
 15  Class_creditcard  434838 non-null  float64
 16  V21               43

In [18]:
# Categorical columns to encode
categorical_columns = ['browser', 'sex', 'country']

# Apply One-Hot Encoding
combined_data = pd.get_dummies(combined_data, columns=categorical_columns, drop_first=True)

# Check the result
combined_data.head()


,purchase_value,age,class_fraud,purchase_month,purchase_day,purchase_hour,V4,V15,V18,V16,V6,V28,Class_creditcard,V21,V17,V23,V7,V25,V1,V10,V24,V14,Amount,V12,V20,V3,V8,V22,V5,V27,V2,V26,V19,V9,V11,V13,browser_FireFox,browser_IE,browser_Opera,browser_Safari,sex_M,country_Albania,country_Algeria,country_Angola,country_Antigua and Barbuda,country_Argentina,country_Armenia,country_Australia,country_Austria,country_Azerbaijan,country_Bahamas,country_Bahrain,country_Bangladesh,country_Barbados,country_Belarus,country_Belgium,country_Belize,country_Benin,country_Bermuda,country_Bhutan,country_Bolivia,country_Bonaire; Sint Eustatius; Saba,country_Bosnia and Herzegowina,country_Botswana,country_Brazil,country_British Indian Ocean Territory,country_Brunei Darussalam,country_Bulgaria,country_Burkina Faso,country_Burundi,country_Cambodia,country_Cameroon,country_Canada,country_Cape Verde,country_Cayman Islands,country_Chile,country_China,country_Colombia,country_Congo,country_Congo The Democratic Republic of The,country_Costa Rica,country_Cote D'ivoire,country_Croatia (LOCAL Name: Hrvatska),country_Cuba,country_Curacao,country_Cyprus,country_Czech Republic,country_Denmark,country_Djibouti,country_Dominica,country_Dominican Republic,country_Ecuador,country_Egypt,country_El Salvador,country_Estonia,country_Ethiopia,country_European Union,country_Faroe Islands,country_Fiji,country_Finland,country_France,country_Gabon,country_Gambia,country_Georgia,country_Germany,country_Ghana,country_Gibraltar,country_Greece,country_Guadeloupe,country_Guam,country_Guatemala,country_Haiti,country_Honduras,country_Hong Kong,country_Hungary,country_Iceland,country_India,country_Indonesia,country_Iran (ISLAMIC Republic Of),country_Iraq,country_Ireland,country_Israel,country_Italy,country_Jamaica,country_Japan,country_Jordan,country_Kazakhstan,country_Kenya,country_Korea Republic of,country_Kuwait,country_Kyrgyzstan,country_Lao People's Democratic Republic,country_Latvia,country_Lebanon,country_Lesotho,country_Libyan Arab Jamahiriya,country_Liechtenstein,country_Lithuania,country_Luxembourg,country_Macau,country_Macedonia,country_Madagascar,country_Malawi,country_Malaysia,country_Maldives,country_Malta,country_Mauritius,country_Mexico,country_Moldova Republic of,country_Monaco,country_Mongolia,country_Montenegro,country_Morocco,country_Mozambique,country_Myanmar,country_Namibia,country_Nauru,country_Nepal,country_Netherlands,country_New Caledonia,country_New Zealand,country_Nicaragua,country_Niger,country_Nigeria,country_Norway,country_Oman,country_Pakistan,country_Palestinian Territory Occupied,country_Panama,country_Papua New Guinea,country_Paraguay,country_Peru,country_Philippines,country_Poland,country_Portugal,country_Puerto Rico,country_Qatar,country_Reunion,country_Romania,country_Russian Federation,country_Rwanda,country_Saint Kitts and Nevis,country_Saint Martin,country_San Marino,country_Saudi Arabia,country_Senegal,country_Serbia,country_Seychelles,country_Singapore,country_Slovakia (SLOVAK Republic),country_Slovenia,country_South Africa,country_South Sudan,country_Spain,country_Sri Lanka,country_Sudan,country_Sweden,country_Switzerland,country_Syrian Arab Republic,country_Taiwan; Republic of China (ROC),country_Tajikistan,country_Tanzania United Republic of,country_Thailand,country_Trinidad and Tobago,country_Tunisia,country_Turkey,country_Turkmenistan,country_Uganda,country_Ukraine,country_United Arab Emirates,country_United Kingdom,country_United States,country_Unknown,country_Uruguay,country_Uzbekistan,country_Vanuatu,country_Venezuela,country_Viet Nam,country_Virgin Islands (U.S.),country_Yemen,country_Zambia,country_Zimbabwe
0,34.0,39.0,0.0,4.0,18.0,2.0,-0.002966,0.001043,0.001515,0.001162,-0.001139,0.000547,0.001667,-0.000371,0.00017,0.000198,0.001801,-0.000232,0.005917,-0.001441,0.000214,0.000252,1.0,-0.000715,0.000187,0.001613,-0.000854,-0.000015,0.001828,0.001763,-0.004135,0.000149,-0.000264,-0.001596,0.000202,0.000603,False,False,False,Fals

In [19]:
boolean_columns = combined_data.select_dtypes(include="bool").columns

combined_data[boolean_columns] = combined_data[boolean_columns].astype(int)

In [20]:
combined_data.head()

,purchase_value,age,class_fraud,purchase_month,purchase_day,purchase_hour,V4,V15,V18,V16,V6,V28,Class_creditcard,V21,V17,V23,V7,V25,V1,V10,V24,V14,Amount,V12,V20,V3,V8,V22,V5,V27,V2,V26,V19,V9,V11,V13,browser_FireFox,browser_IE,browser_Opera,browser_Safari,sex_M,country_Albania,country_Algeria,country_Angola,country_Antigua and Barbuda,country_Argentina,country_Armenia,country_Australia,country_Austria,country_Azerbaijan,country_Bahamas,country_Bahrain,country_Bangladesh,country_Barbados,country_Belarus,country_Belgium,country_Belize,country_Benin,country_Bermuda,country_Bhutan,country_Bolivia,country_Bonaire; Sint Eustatius; Saba,country_Bosnia and Herzegowina,country_Botswana,country_Brazil,country_British Indian Ocean Territory,country_Brunei Darussalam,country_Bulgaria,country_Burkina Faso,country_Burundi,country_Cambodia,country_Cameroon,country_Canada,country_Cape Verde,country_Cayman Islands,country_Chile,country_China,country_Colombia,country_Congo,country_Congo The Democratic Republic of The,country_Costa Rica,country_Cote D'ivoire,country_Croatia (LOCAL Name: Hrvatska),country_Cuba,country_Curacao,country_Cyprus,country_Czech Republic,country_Denmark,country_Djibouti,country_Dominica,country_Dominican Republic,country_Ecuador,country_Egypt,country_El Salvador,country_Estonia,country_Ethiopia,country_European Union,country_Faroe Islands,country_Fiji,country_Finland,country_France,country_Gabon,country_Gambia,country_Georgia,country_Germany,country_Ghana,country_Gibraltar,country_Greece,country_Guadeloupe,country_Guam,country_Guatemala,country_Haiti,country_Honduras,country_Hong Kong,country_Hungary,country_Iceland,country_India,country_Indonesia,country_Iran (ISLAMIC Republic Of),country_Iraq,country_Ireland,country_Israel,country_Italy,country_Jamaica,country_Japan,country_Jordan,country_Kazakhstan,country_Kenya,country_Korea Republic of,country_Kuwait,country_Kyrgyzstan,country_Lao People's Democratic Republic,country_Latvia,country_Lebanon,country_Lesotho,country_Libyan Arab Jamahiriya,country_Liechtenstein,country_Lithuania,country_Luxembourg,country_Macau,country_Macedonia,country_Madagascar,country_Malawi,country_Malaysia,country_Maldives,country_Malta,country_Mauritius,country_Mexico,country_Moldova Republic of,country_Monaco,country_Mongolia,country_Montenegro,country_Morocco,country_Mozambique,country_Myanmar,country_Namibia,country_Nauru,country_Nepal,country_Netherlands,country_New Caledonia,country_New Zealand,country_Nicaragua,country_Niger,country_Nigeria,country_Norway,country_Oman,country_Pakistan,country_Palestinian Territory Occupied,country_Panama,country_Papua New Guinea,country_Paraguay,country_Peru,country_Philippines,country_Poland,country_Portugal,country_Puerto Rico,country_Qatar,country_Reunion,country_Romania,country_Russian Federation,country_Rwanda,country_Saint Kitts and Nevis,country_Saint Martin,country_San Marino,country_Saudi Arabia,country_Senegal,country_Serbia,country_Seychelles,country_Singapore,country_Slovakia (SLOVAK Republic),country_Slovenia,country_South Africa,country_South Sudan,country_Spain,country_Sri Lanka,country_Sudan,country_Sweden,country_Switzerland,country_Syrian Arab Republic,country_Taiwan; Republic of China (ROC),country_Tajikistan,country_Tanzania United Republic of,country_Thailand,country_Trinidad and Tobago,country_Tunisia,country_Turkey,country_Turkmenistan,country_Uganda,country_Ukraine,country_United Arab Emirates,country_United Kingdom,country_United States,country_Unknown,country_Uruguay,country_Uzbekistan,country_Vanuatu,country_Venezuela,country_Viet Nam,country_Virgin Islands (U.S.),country_Yemen,country_Zambia,country_Zimbabwe
0,34.0,39.0,0.0,4.0,18.0,2.0,-0.002966,0.001043,0.001515,0.001162,-0.001139,0.000547,0.001667,-0.000371,0.00017,0.000198,0.001801,-0.000232,0.005917,-0.001441,0.000214,0.000252,1.0,-0.000715,0.000187,0.001613,-0.000854,-0.000015,0.001828,0.001763,-0.004135,0.000149,-0.000264,-0.001596,0.000202,0.000603,0,0,0,0,1,0,0,0,0,0,0,

In [21]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd

# 1. Filter numeric columns
numeric_columns = combined_data.select_dtypes(include=['float64', 'int64']).columns

# 2. Initialize the StandardScaler
scaler = StandardScaler()

# 3. Fit and transform only the numeric columns
scaled_data = scaler.fit_transform(combined_data[numeric_columns])

# 4. Create a DataFrame with the scaled numeric columns
scaled_data_df = pd.DataFrame(scaled_data, columns=numeric_columns)

# 5. Replace the original numeric columns in `combined_data` with the scaled columns
combined_data[numeric_columns] = scaled_data_df

# Now all the numeric columns in combined_data are standardized


In [22]:
scaler_MinMax = MinMaxScaler()
combined_data[numerical_columns] = scaler.fit_transform(combined_data[numerical_columns])

In [23]:
combined_data.sample(7)

,purchase_value,age,class_fraud,purchase_month,purchase_day,purchase_hour,V4,V15,V18,V16,V6,V28,Class_creditcard,V21,V17,V23,V7,V25,V1,V10,V24,V14,Amount,V12,V20,V3,V8,V22,V5,V27,V2,V26,V19,V9,V11,V13,browser_FireFox,browser_IE,browser_Opera,browser_Safari,sex_M,country_Albania,country_Algeria,country_Angola,country_Antigua and Barbuda,country_Argentina,country_Armenia,country_Australia,country_Austria,country_Azerbaijan,country_Bahamas,country_Bahrain,country_Bangladesh,country_Barbados,country_Belarus,country_Belgium,country_Belize,country_Benin,country_Bermuda,country_Bhutan,country_Bolivia,country_Bonaire; Sint Eustatius; Saba,country_Bosnia and Herzegowina,country_Botswana,country_Brazil,country_British Indian Ocean Territory,country_Brunei Darussalam,country_Bulgaria,country_Burkina Faso,country_Burundi,country_Cambodia,country_Cameroon,country_Canada,country_Cape Verde,country_Cayman Islands,country_Chile,country_China,country_Colombia,country_Congo,country_Congo The Democratic Republic of The,country_Costa Rica,country_Cote D'ivoire,country_Croatia (LOCAL Name: Hrvatska),country_Cuba,country_Curacao,country_Cyprus,country_Czech Republic,country_Denmark,country_Djibouti,country_Dominica,country_Dominican Republic,country_Ecuador,country_Egypt,country_El Salvador,country_Estonia,country_Ethiopia,country_European Union,country_Faroe Islands,country_Fiji,country_Finland,country_France,country_Gabon,country_Gambia,country_Georgia,country_Germany,country_Ghana,country_Gibraltar,country_Greece,country_Guadeloupe,country_Guam,country_Guatemala,country_Haiti,country_Honduras,country_Hong Kong,country_Hungary,country_Iceland,country_India,country_Indonesia,country_Iran (ISLAMIC Republic Of),country_Iraq,country_Ireland,country_Israel,country_Italy,country_Jamaica,country_Japan,country_Jordan,country_Kazakhstan,country_Kenya,country_Korea Republic of,country_Kuwait,country_Kyrgyzstan,country_Lao People's Democratic Republic,country_Latvia,country_Lebanon,country_Lesotho,country_Libyan Arab Jamahiriya,country_Liechtenstein,country_Lithuania,country_Luxembourg,country_Macau,country_Macedonia,country_Madagascar,country_Malawi,country_Malaysia,country_Maldives,country_Malta,country_Mauritius,country_Mexico,country_Moldova Republic of,country_Monaco,country_Mongolia,country_Montenegro,country_Morocco,country_Mozambique,country_Myanmar,country_Namibia,country_Nauru,country_Nepal,country_Netherlands,country_New Caledonia,country_New Zealand,country_Nicaragua,country_Niger,country_Nigeria,country_Norway,country_Oman,country_Pakistan,country_Palestinian Territory Occupied,country_Panama,country_Papua New Guinea,country_Paraguay,country_Peru,country_Philippines,country_Poland,country_Portugal,country_Puerto Rico,country_Qatar,country_Reunion,country_Romania,country_Russian Federation,country_Rwanda,country_Saint Kitts and Nevis,country_Saint Martin,country_San Marino,country_Saudi Arabia,country_Senegal,country_Serbia,country_Seychelles,country_Singapore,country_Slovakia (SLOVAK Republic),country_Slovenia,country_South Africa,country_South Sudan,country_Spain,country_Sri Lanka,country_Sudan,country_Sweden,country_Switzerland,country_Syrian Arab Republic,country_Taiwan; Republic of China (ROC),country_Tajikistan,country_Tanzania United Republic of,country_Thailand,country_Trinidad and Tobago,country_Tunisia,country_Turkey,country_Turkmenistan,country_Uganda,country_Ukraine,country_United Arab Emirates,country_United Kingdom,country_United States,country_Unknown,country_Uruguay,country_Uzbekistan,country_Vanuatu,country_Venezuela,country_Viet Nam,country_Virgin Islands (U.S.),country_Yemen,country_Zambia,country_Zimbabwe
43166,-7.346696e-01,-1.011918e+00,-5.452672e-01,6.320715e-01,-8.234325e-01,-2.582044e+00,-7.134193e-18,-4.005956e-17,-1.474997e-17,3.209412e-18,-8.148484e-18,5.909070e-18,1.209817e-17,1.292892e-17,-1.077146e-17,1.588247e-18,-8.670515e-18,1.525674e-17,9.547124e-18,-3.008336e-18,-8.637411e-18,-2.757753e-18,-0.276380,2.475714e-

In [24]:
combined_data.isnull().sum().sum()

np.int64(0)